# sklearn-LDA

代码示例：https://mp.weixin.qq.com/s/hMcJtB3Lss1NBalXRTGZlQ （玉树芝兰） <br>
可视化：https://blog.csdn.net/qq_39496504/article/details/107125284  <br>
sklearn lda参数解读:https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
<br>中文版参数解读：https://blog.csdn.net/TiffanyRabbit/article/details/76445909
<br>LDA原理-视频版：https://www.bilibili.com/video/BV1t54y127U8
<br>LDA原理-文字版：https://www.jianshu.com/p/5c510694c07e
<br>score的计算方法：https://github.com/scikit-learn/scikit-learn/blob/844b4be24d20fc42cc13b957374c718956a0db39/sklearn/decomposition/_lda.py#L729
<br>主题困惑度1：https://blog.csdn.net/weixin_43343486/article/details/109255165
<br>主题困惑度2：https://blog.csdn.net/weixin_39676021/article/details/112187210

## 0.导包

In [33]:
import pandas as pd
import re
import jieba
import jieba.posseg as psg
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

## 1.预处理

In [34]:
class Prepare:
    def __init__(self):
        # 读专有名词和禁止词的词典
        ## 专有名词
        jieba.load_userdict(r"C:\Users\yerdon\hustzz\hustzz\LDA\stop_dic\dict.txt")   # 专有词汇词典
        jieba.initialize()

        ## 禁止词
        try:
            stopword_list = open(r"C:\Users\yerdon\hustzz\hustzz\LDA\stop_dic\stopwords.txt", encoding ='utf-8')     # 禁止词（没必要被分析的）
            self.stop_list = []
            for stopword in stopword_list:
                self.stop_list.append(re.sub(u'\n|\\r', '', stopword))
        except:
            stopword_list = []
            print("检查禁止词文件的格式")
        
    
    def chinese_word_cut(self, mytext):
        """
        输入文本，返回被拆分好的文本
        @mytext: 从data表格的每一个格子里面输入的被分析的文本
        """
        word_list = []
        # jieba分词
        for seg_word in psg.cut(mytext):
            word = re.sub(u'[^\u4e00-\u9fa5]', '', seg_word.word)       # 用正则表达式挑选中文词汇
            # 三重条件筛选词汇，其一长度需要大于等于2，其二须是名词、专有名词或动名词，其三，不在禁止词之列
            if len(word) >= 2 and seg_word.flag in ['n', 'nz', 'vn'] and not word in self.stop_list:
                word_list.append(word)
        return (" ").join(word_list)
    
    @staticmethod
    def get_tf(data_serie, n_features=1000):
        """input cutted words and n_features, return tf

        Args:
            data_serie (pd.series): refer to data['content_cutted'], actually. 
            n_features (int, optional): Total amount of featured words, according to volume of texts. Defaults to 1000.

        Returns:
            tf: used for lda
        """
        tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                        max_features=n_features,
                                        stop_words='english',
                                        max_df = 0.5,
                                        min_df = 10)
        return [tf_vectorizer, tf_vectorizer.fit_transform(data_serie)]
    
    def main(self):
        data = pd.read_excel(r'C:\Users\yerdon\hustzz\hustzz\LDA\data\data.xls')     # 数据
        data["content_cutted"] = data['content'].apply(self.chinese_word_cut)
        print(data)
        return data

## 2.应该先算困惑度

In [35]:
class Perplexity:
    def __init__(self, n_features=1000):
        pre = Prepare()
        self.data = pre.main()
        self.tf = Prepare.get_tf(self.data['content_cutted'], n_features)[1]
        
    def get_min_plx(self):
        plx = []
        for i in range(1, 50):
            lda = LatentDirichletAllocation(n_components=i, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,random_state=0)
            lda.fit(self.tf)
            plx.append(lda.perplexity(self.tf))
        
        return plx.index(min(plx)) + 1      # 返回困惑度最小的主题数量   

## 3.LDA分析

In [36]:
class LDA:
    def __init__(self, n_features=1000):
        """training model with the given amounts of topics and featured words

        Args:
            n_topics (int, optional): Total amount of topics, according to complexity. Defaults to 8.
            n_features (int, optional): Total amount of featured words, according to volume of texts. Defaults to 1000.
        """
        pre = Prepare()
        plx = Perplexity()
        n_topics = plx.get_min_plx()
        self.data = pre.main()
        self.lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                    learning_method='batch',
                                    learning_offset=50,
                                    # doc_topic_prior=0.1,
                                    # topic_word_prior=0.01,
                                    random_state=0)
        self.tf = Prepare.get_tf(self.data['content_cutted'], n_features)[1]
        self.tf_vectorizer = Prepare.get_tf(self.data['content_cutted'], n_features)[0]
        self.lda.fit(self.tf)
        
        pyLDAvis.enable_notebook()
        pic = pyLDAvis.sklearn.prepare(self.lda, self.tf, self.tf_vectorizer)
        pyLDAvis.save_html(pic, 'lda_pass'+ str(n_topics)+'.html')


    def get_topic(self):
        topics = self.lda.transform(self.tf)
        
        topic = []
        for t in topics:
            topic.append("Topic #" + str(list(t).index(np.max(t))))
        self.data['概率最大的主题序号'] = topic
        # self.data['每个主题对应概率'] = list(topics)
        self.data.to_excel(r'C:\Users\yerdon\hustzz\hustzz\LDA\res\data.xls', index=False)
        

In [37]:
LDA()

                                              content  \
0   省人民政府关于促进大数据云计算人工智能创新发展加快建设数字贵州的意见\n各市、自治州人民政府...   
1   广西推进数字政府建设三年行动计划\n（2018—2020年）\n根据《中共广西壮族自治区委员...   
2   广东省“数字政府”建设总体规划\n（2018-2020年）实施方案\n\n为贯彻落实我省“数...   
3   浙江省深化“最多跑一次”改革推进政府数字化转型工作总体方案\n\n为深入贯彻落实《浙江省深化...   
4   山东省数字政府建设实施方案（2019-2022年）为深入贯彻落实《数字山东发展规划（2018...   
5   山西省加快数字政府建设实施方案，为深入贯彻落实党中央、国务院关于加快数字中国建设的决策部署,...   
6   河北省数字经济发展规划\n（2020-2025年）\n　　数字经济是继农业经济、工业经济之后...   
7   湖北省数字政府建设总体规划（2020-2022年）\n数字政府建设是落实网络强国战略、加快建...   
8   安徽省“数字政府”建设规划（2020—2025年）“数字政府”建设是以新一代信息技术为支撑，...   
9   推动电子信息产业和数字政府建设促进“数字吉林”快速发展工作方案（2020—2025年）\n按...   
10  河南省数字政府建设总体规划（2020—2022年）\n为贯彻落实党中央、国务院关于加快推进“...   
11  为加快推动“十四五”时期北京智慧城市发展，依据《北京城市总体规划(2016年—2035年)》...   
12  宁夏回族自治区数字政府建设行动计划（2021年-2023年）\n \n为贯彻落实习近平总书记...   
13  天津市推进智慧城市建设行动计划（2015—2017年）智慧城市是当今世界城市发展的新趋势，是...   
14  江苏省“十四五”数字政府建设规划\n \n党的十九届五中全会明确要求“加快数字社会、数字政府...   
15  数字政府建设以新一代信息技术为支撑，通过数据驱动政务信息化管理架构、业务架构和技术架构重塑，...   
16  上海市全面推进城市数字化转型“十四五”规划\n  全面

d:\Conda\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
